In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# <<=== GANTI JIKA PERLU
PROJECT_DIR = r"F:\UGM\Tugas\Temu Kembali Informasi\Tugas 4"
CORPUS_SRC  = r"data\finance_jsonl\corpus.jsonl"

print("PROJECT_DIR:", PROJECT_DIR)
print("CORPUS_SRC :", CORPUS_SRC)

PROJECT_DIR: F:\UGM\Tugas\Temu Kembali Informasi\Tugas 4
CORPUS_SRC : data\finance_jsonl\corpus.jsonl


In [2]:
!java --version

openjdk 21.0.8 2025-07-15 LTS
OpenJDK Runtime Environment Zulu21.44+17-CA (build 21.0.8+9-LTS)
OpenJDK 64-Bit Server VM Zulu21.44+17-CA (build 21.0.8+9-LTS, mixed mode, sharing)


In [15]:
# bersihkan index lama
!rm -rf indexes/idx_contents indexes/hybird/idx_contents indexes/idx_hybrid

# A) contents → pakai file asli di data/finance_jsonl
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input data/finance_jsonl \
  --index indexes/idx_contents \
  --generator DefaultLuceneDocumentGenerator \
  --threads 2 \
  --storePositions --storeDocvectors --storeRaw

'rm' is not recognized as an internal or external command,
operable program or batch file.


2025-10-22 07:44:16,656 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:205) - Setting log level to INFO
2025-10-22 07:44:16,662 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - ============ Loading Index Configuration ============
2025-10-22 07:44:16,663 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:209) - AbstractIndexer settings:
2025-10-22 07:44:16,664 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:210) -  + DocumentCollection path: data/finance_jsonl
2025-10-22 07:44:16,664 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) -  + CollectionClass: JsonCollection
2025-10-22 07:44:16,665 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) -  + Index path: indexes/idx_contents
2025-10-22 07:44:16,667 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + Threads: 2
2025-10-22 07:44:16,668 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:214) -  + Optimize (merge segments)? false
2025-10-22 07:44:16,75

Oct 22, 2025 7:44:16 AM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


In [2]:
from pyserini.search.lucene import LuceneSearcher
import json

searcher = LuceneSearcher("indexes/idx_contents")

def search(query, k=5):
    hits = searcher.search(query, k=k)
    results = []
    for i, h in enumerate(hits, 1):
        raw = searcher.doc(h.docid).raw()
        try:
            obj = json.loads(raw)
            title = obj.get("title","").strip()
            url   = obj.get("url","")
            snippet = obj.get("contents","")[:300].replace("\n"," ")
        except Exception:
            title, url, snippet = "", "", ""
        results.append({
            "rank": i, "docid": h.docid, "score": h.score,
            "title": title, "url": url, "snippet": snippet
        })
    return results

c:\Users\audih\miniconda3\envs\tki\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
queries = [
    "pergerakan indeks harga saham gabungan ihsg"
]

for q in queries: 
    print(f"\n=== Query: {q} ===")
    for r in search(q, k=10):
        print(f"{r['rank']}. {r['title']} (score={r['score']:.2f})")
        if r['url']:
            print("   ", r['url'])


=== Query: pergerakan indeks harga saham gabungan ihsg ===
1. ihsg potensi lanjut kuat rabu 3 9 2025 simak rekomendasi saham (score=3.29)
    https://investasi.kontan.co.id/news/ihsg-berpotensi-lanjut-menguat-rabu-392025-simak-rekomendasi-sahamnya
2. ihsg mulai rebound pasca reshuffle menteri riskan koreksi (score=3.26)
    https://investasi.kontan.co.id/news/ihsg-mulai-rebound-pasca-reshuffle-menteri-masih-riskan-terkoreksi
3. bukan blue chip harga saham second liner terus daki cek layak beli jual (score=3.22)
    https://investasi.kontan.co.id/news/bukan-blue-chip-harga-saham-second-liner-terus-mendaki-cek-yang-layak-beli-jual
4. saham arto lompat 7 11 jadi top gainers utama lq45 sentimen positif (score=3.17)
    https://investasi.kontan.co.id/news/saham-arto-melompat-711-dan-jadi-top-gainers-utama-lq45-ini-sentimen-positifnya
5. kuat rapuh lonjak ihsg sokong emiten market cap jumbo (score=3.06)
    https://investasi.kontan.co.id/news/penguatan-masih-rapuh-lonjakan-ihsg-hanya-disoko